# Lasagne basics

## Build LeNet network for MNIST

In [1]:
# Import layers and softmax from Lasagne
import theano
import lasagne

# x (input) and y (target) variables
x_var = theano.tensor.tensor4('x')
y_var = theano.tensor.ivector('y')

# Input layer, un-constrained mini-batch size, 1 channel, 28 x 28
l_in = lasagne.layers.InputLayer(shape=(None, 1, 28, 28), input_var=x_var)
# First convolutional layer, 20 filters, 5x5 filter shape
l_c1 = lasagne.layers.Conv2DLayer(l_in, num_filters=20, filter_size=(5,5))
# Max-pooling layer, 2x2 pool size
l_p1 = lasagne.layers.MaxPool2DLayer(l_c1, pool_size=(2,2))
# Second convolutional layer, 50 filters, 5x5 filter shape
l_c2 = lasagne.layers.Conv2DLayer(l_p1, num_filters=50, filter_size=(5,5))
# Max-pooling layer, 2x2 pool size
l_p2 = lasagne.layers.MaxPool2DLayer(l_c2, pool_size=(2,2))
# Dense layer, 256 units
l_d3 = lasagne.layers.DenseLayer(l_p2, num_units=256)
# 50% dropout
l_d3p = lasagne.layers.DropoutLayer(l_d3, p=0.5)
# Final softmax dense layer, 10 units
l_final = lasagne.layers.DenseLayer(l_d3p, num_units=10, nonlinearity=lasagne.nonlinearities.softmax)

EVIL HACK: Disable cuDNN check


DEBUG: nvcc STDOUT mod.cu
   Creating library D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpwvma2e/265abc51f7c376c224983485238ff1a5.lib and object D:/temp/theano/compiledir_Windows-10-10.0.10586-Intel64_Family_6_Model_58_Stepping_9_GenuineIntel-2.7.11-64/tmpwvma2e/265abc51f7c376c224983485238ff1a5.exp

Using gpu device 0: GeForce GTX 970 (CNMeM is enabled with initial size: 25.0% of memory, cuDNN 4007)


## Loss, parameters, updates, train and evaluation functions

In [2]:
train_pred_prob = lasagne.layers.get_output(l_final)
train_loss = lasagne.objectives.categorical_crossentropy(train_pred_prob, y_var)

params = lasagne.layers.get_all_params(l_final, trainable=True)
updates = lasagne.updates.nesterov_momentum(train_loss.mean(), params, learning_rate=0.01, momentum=0.9)

eval_pred = lasagne.layers.get_output(l_final, deterministic=True)
eval_loss = lasagne.objectives.categorical_crossentropy(eval_pred, y_var)

train_fn = theano.function([x_var, y_var], train_loss.sum(), updates=updates)
eval_fn = theano.function([x_var, y_var], eval_loss.sum())
pred_prob_fn = theano.function([x_var], eval_pred)


## Train the network

In [3]:
import numpy as np
import mnist_dataset, fuel

mnist = mnist_dataset.MNISTTrainValTest()
train_scheme = fuel.schemes.ShuffledScheme(examples=mnist.train.num_examples, batch_size=128)
# Use `DataStream.default_stream`, otherwise the default transformers defined by the dataset *wont* be applied
train_stream = fuel.streams.DataStream.default_stream(dataset=mnist.train, iteration_scheme=train_scheme)

for epoch in range(25):
    tr_loss = 0.0
    train_stream.reset()
    for b_x, b_y in train_stream.get_epoch_iterator():
        tr_loss += train_fn(b_x, b_y[:,0])
    tr_loss /= float(mnist.train.num_examples)
    print('Epoch {} train loss {:.5f}'.format(epoch, tr_loss))

Epoch 0 train loss 0.43908
Epoch 1 train loss 0.12628
Epoch 2 train loss 0.08842
Epoch 3 train loss 0.07258
Epoch 4 train loss 0.06142
Epoch 5 train loss 0.05562
Epoch 6 train loss 0.05125
Epoch 7 train loss 0.04520
Epoch 8 train loss 0.04101
Epoch 9 train loss 0.03547
Epoch 10 train loss 0.03502
Epoch 11 train loss 0.03142
Epoch 12 train loss 0.03073
Epoch 13 train loss 0.02753
Epoch 14 train loss 0.02542
Epoch 15 train loss 0.02340
Epoch 16 train loss 0.02281
Epoch 17 train loss 0.02189
Epoch 18 train loss 0.02032
Epoch 19 train loss 0.02020
Epoch 20 train loss 0.01725
Epoch 21 train loss 0.01766
Epoch 22 train loss 0.01606
Epoch 23 train loss 0.01560
Epoch 24 train loss 0.01537


## Evaluate the network

In [5]:
test_scheme = fuel.schemes.SequentialScheme(examples=mnist.test.num_examples, batch_size=128)
# Use `DataStream.default_stream`, otherwise the default transformers defined by the dataset *wont* be applied
test_stream = fuel.streams.DataStream.default_stream(dataset=mnist.test, iteration_scheme=test_scheme)
test_err = 0.0
for b_x, b_y in test_stream.get_epoch_iterator():
    b_pred_prob = pred_prob_fn(b_x)
    b_pred_cls = np.argmax(b_pred_prob, axis=1)
    test_err += (b_y[:,0] != b_pred_cls).sum()
test_err /= float(mnist.test.num_examples)
print('Final err {:.2%}'.format(test_err))

    

Final err 0.60%
